# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('weatherdata.csv')
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vao,-22.6667,167.4833,66.74,63,0,13.96,NC,1631903887
1,1,Mahébourg,-20.4081,57.7000,72.25,94,40,6.91,MU,1631903888
2,2,Ostrovnoy,68.0531,39.5131,41.99,73,100,5.86,RU,1631903889
3,3,Albany,42.6001,-73.9662,76.08,79,78,6.91,US,1631903865
4,4,Port Alfred,-33.5906,26.8910,58.12,58,71,4.70,ZA,1631903890


In [3]:
gmaps.configure(api_key=g_key)

locations = weather_data[['Lat','Lng']]
humidities = weather_data['Humidity']


In [4]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Ideal Conditions
low = 70
high = 80
wind_speed_max = 10
cloudiness_max = 0

ideal_data = weather_data[(weather_data['Wind Speed'] <= wind_speed_max) & (weather_data['Cloudiness'] <= cloudiness_max) & (weather_data['Max Temp'] <= high)& (weather_data['Max Temp'] >= low)]
ideal_data = ideal_data.dropna()

ideal_data.head(20)





,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,133,Elko,41.0002,-115.5012,75.74,26,0,3.00,US,1631903962
137,137,Kiryat Gat,31.6100,34.7642,78.82,100,0,6.06,IL,1631903963
145,145,Saint-Pierre,-21.3393,55.4781,71.28,75,0,5.75,RE,1631903967
170,170,Cutro,39.0353,16.9819,78.03,53,0,9.22,IT,1631903982
190,190,Erzin,36.9559,36.2000,77.00,67,0,2.62,TR,1631903995
214,214,Karratha,-20.7377,116.8463,71.02,56,0,4.63,AU,1631904012
225,225,Saint-Leu,-21.1500,55.2833,70.66,78,0,5.21,RE,1631904020
233,233,Saint-Philippe,-21.3585,55.7679,71.19,77,0,7.87,RE,1631904026
242,242,Yangi Marg`ilon,40.4272,71.7189,70.52,43,0,9.22,UZ,1631904032
273,273,Shenjiamen,29.9576,122.2980,72.32,73,0,2.86,CN,1631903886


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = ideal_data
hotel_df['Hotel Name'] = 'error: none found'

target_type = 'lodging'
radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    location = f"{row['Lat']},{row['Lng']}"
    params ={
        'location':location,
        'type':target_type,
        'radius':radius,
        'key':g_key
    }
    
    
    response = requests.get(base_url, params).json()
    try:
        hotel_name = response["results"][0]["name"]
        print(f"City: {row['City']}")
        print(f'___________{hotel_name}___________________')
        print('________________________________________')
        
        hotel_df.loc[index, ['Hotel Name']] = hotel_name
       
 
        
    except:
        print(f"City: {row['City']}")
        print('____________error: none found________________')
        print('________________________________________')
        pprint(response)
        





City: Elko
____________error: none found________________
________________________________________
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
City: Kiryat Gat
___________OlusHome___________________
________________________________________
City: Saint-Pierre
___________Lindsey Hôtel___________________
________________________________________
City: Cutro
___________Gallella Viaggi - Agenzia Viaggi e Turismo___________________
________________________________________
City: Erzin
___________Artemis Otel___________________
________________________________________
City: Karratha
___________ibis Styles Karratha___________________
________________________________________
City: Saint-Leu
___________La Caz Oceane___________________
________________________________________
City: Saint-Philippe
___________Chambres d'hôte "La Trinité"___________________
________________________________________
City: Yangi Marg`ilon
___________Ikathouse___________________
_____________________

In [17]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
133,133,Elko,41.0002,-115.5012,75.74,26,0,3.00,US,1631903962,error: none found
137,137,Kiryat Gat,31.6100,34.7642,78.82,100,0,6.06,IL,1631903963,OlusHome
145,145,Saint-Pierre,-21.3393,55.4781,71.28,75,0,5.75,RE,1631903967,Lindsey Hôtel
170,170,Cutro,39.0353,16.9819,78.03,53,0,9.22,IT,1631903982,Gallella Viaggi - Agenzia Viaggi e Turismo
190,190,Erzin,36.9559,36.2000,77.00,67,0,2.62,TR,1631903995,Artemis Otel
214,214,Karratha,-20.7377,116.8463,71.02,56,0,4.63,AU,1631904012,ibis Styles Karratha
225,225,Saint-Leu,-21.1500,55.2833,70.66,78,0,5.21,RE,1631904020,La Caz Oceane
233,233,Saint-Philippe,-21.3585,55.7679,71.19,77,0,7.87,RE,1631904026,"Chambres d'hôte ""La Trinité"""
242,242,Yangi Marg`ilon,40.4272,71.7189,70.52,43,0,9.22,UZ,1631904032,Ikathouse
273,273,Shenjiamen,29.9576,122.2980,72.32,73,0,2.86,CN,1631903886,Jinjiang Inn Zhoushan Putuo Shenjiamen


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
# Create symbol layer
layer = gmaps.marker_layer(
    locations, 
    info_box_content=hotel_info
)

# Display figure

fig.add_layer(layer)

fig


Figure(layout=FigureLayout(height='420px'))